In [2]:
import os
import openai
import sys
import pandas as pd
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain


openai.api_key  = os.environ['OPENAI_API_KEY']


from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)

In [10]:
df = pd.DataFrame(
    [
        {'case_id':355263,"ocr":'z.i. iIIII- 1:; . LRTA.Un. g3-33 2995277 NIP: 6Błar: . Nr . ISJRÓYRMIAGYJNA IZBY PRZYJIć . Nazyiskć xAdres: . Dnia iodlz. przyjęcia: . 2020-04-10 15:17 . SPESEL: . Data urodzenia: . Pata iąodz wyypisu: 2020-04-10 15:40 . ŚOZPOZNANZA . SŚ.1 Otwarta rana palca Cpalców) z uszkodzeniem paznokcia . Gliwice, dnia OSod . Pacjent zgłosił się z powodu rany palca I ręki prawej - Rana powstała W wyniku kontaktu z piłą mechaniczng. . BADANIE PODMIOTOWE . Rana opuszki palca I ręki prawej wraz z częściowym uszkodzeniem paznokcia. w znieczulenfu miejscowym szycie rany   opatrunck, anatoksyna . Dystalna ruchomośc , ukrwienie i unerwienie zachowane, pełne a . K8rtrola w Poradni Cnirurgicznej za dni . BADANIE IPRZRDMIOTOWE . WYNIRCI BADAŃ LABORATORYJNYCH . WXINIIKI BADA ORRAZOWYCE I INNYCHI . WXNIKI K.OISUILTACJI . ZASTOSOWANIE LBCZBNIE . DAFILON 3/0, . ZALIECENIA /POWOD ODMOWX . Cpieczątka i pbapis ŁŁ Zekarza) ssssaczoinej Sonczar'},
        {'case_id':355278,"ocr":'. OPIS BADANIA RADIOLOGICZNEGO . NZOZ pronmm diagnostyka obrazowa . Zespół Opieki zdrowotnej we VVłOSZCZzczowie ul. Zeromskiego 28, 29-100 VVTOSZC:zczowa . Nazwisko: . Lekarz Kierujący: I Szczecinska Ewa . Radlolog: Manikowski dacek . NrF Pacjenta: 19622 irię . Viek: . PESEL: . Data badania: 2020-02-10 . Oddział: . Badany organ zdj- sstawu skokowego AP + profil dednostka kierująca: ZOZ OSZO:zozowa . Złamanie KOStKI tylnej ) poza tym be zmian urazowych . -d/sokcMik Mnelayibox akcc(/aowekz . ZO45a71 ata wykonania opisu: 2020-02-13 . Podpis: [NEXT_PAGE] *U) wls. 7u-100 zadrowoinej wo WIoaxniowia Szpstal Powiatowy n. .0 1) raoel RŁCSON ia- . Nr ks.gł.: 01560/2020 . Karta informacyina   ieczenia szpitalnego . Nunner KS. od 694/2020 . Ko res- 000000014598 Vcz.o1 wIl cz.o76 VIII cz.4902 . Szpłtalny Oddziaf Raturkowy tel. fax- (041)3883707 . Nazwisko i imię: . Adres: . Pacjent w szpitalu od 14-02-2020 do . Rozncznaniei . Urodzony: . PESEL: . Tr dok. ubezp.: . S89.8 d Inne określone urazy poduclzia . WYWIAD . Zgłosit się na SOR z powodu urazu stawu skokowego prawcgo 1.02.2020 twicrdzi - ze był na sor w . OPIS KIBLCAch, BADANJLECZENIE 1 nie został zaratwionyl.Dotychczns zdrowyl,w kontakcie GCS 1-90P- . ADNOTACJI KONSUTTACJTE . Badanie lekarskie, orlOpedyczna: uuraz stawu skokowego prawego tyg tenu umlarkowany obrzęk, ruchomość doś6 dobra, LóI W skrajnym zakresie proponuję unieruchomienie w szynie gipsowej, zakaz obciążania, IIDCz złamanie niewielkiego frajzmentu krawędzi tylnej piszczeli kontrola VV poradni ortopedycznej za tyg . Wyniki bad. obrazowych 13-02-2020: Zdj- stawu skokowego AP   profi1 . Złamanie kostki tylnej > poza tymn bez zmian urazowych 13-02-2020: Zdj. stopy AP + profńl . Stopa bez zmian urazowych . Zlamanie kostki tylnej kości piszczelowej . ZALIBCICNIA . Kontrola i leczenie jak zalecił Ortopeda clexane 1x 40 mg S /c kwilu/ę ocdbidr kurige wyAzziecyjme/ .dlokumeniacy/: ricod. dlostarczonej ra cza.s łowputalłzecyl: (eke, spedjalista. rłoc. J/r swłrurg E-S7- dziecięeł-.   ..- - 58Łn-56 Kitece, u1. Acuo uo AJxwzorakie alzieścez olog : 2Z/5 . I8xeMxowypisuyąey ęi 4 e8753.2 dlcta poalpis . ZOZ Włoszczowa - Szpitalny Oddział Ratunkowy . Eskulap Ruch chorych ver.4.s.ox . Strona 1 1 . Wydrukowane 14-02-2020 14:27 [NEXT_PAGE] 1aataa r 29-09-1975 3 . Nazwisko inię . Numner kolejny wkł. . Poradnia Poradnia Urtopeayczna bata. W1Zy wywiad, badanie pgedplotowe;alecony,+ badania alagniostyczangjkonsultącie, objawy,rozpoznanie,, leczenie.. iwagl.l - podpis 1ekarza   . JGP: . Taryfa: 28-02-2020 . Rozpoznanic zaSadnICZC: . S82.8 Złananie innych ckęści podudzia . Rozpoznaaie wspalistale[3ce: . Opis wizyty: Pacjent po zaopalrzcnru W SOR, do poradni wszedł bez kul. Zdjeto gipso rts kontrolnc jak poprednio. Wniosek ma stabilizator, chodzenie O Kulach:. Zal 29-02 do 3-04-2020. . Skierowania zaświadczenia . ICD9: 89.00 Porada lekarska, konsultacja, asystaj 88.291 RTG kości kończyny dolnej celowane lub czynaościowe . Lekizrecepty: . Strona 111 . Lekarz wykonuący Grzegorz ut r stat. 8735552'}
    ]
)
df[:2]

,case_id,ocr
0,355263,z.i. iIIII- 1:; . LRTA.Un. g3-33 2995277 NIP: ...
1,355278,. OPIS BADANIA RADIOLOGICZNEGO . NZOZ pronmm d...


## podstawowy Example dodawany pomiedzy prefix i suffix

In [11]:
from langchain import FewShotPromptTemplate

#https://colab.research.google.com/github/pinecone-io/examples/blob/master/generation/langchain/handbook/01-langchain-prompt-templates.ipynb#scrollTo=U1LLEbtucipr

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["knowledge"],
    template="- {knowledge}"
)

# create our examples
knowledge_lst = [
    {"knowledge" : "Człowiek możne mieć jedną rękę lewą i jedną rękę prawą."}, 
    {"knowledge" : "Dokładny numer palca zapisuje się cyframi rzymskimi np. I, II , III, IV, V lub i, ii , iii, iv, v."},
    {"knowledge" : "Przykładowa lokalizacja palca to: <uraz> palca <cyfra rzymska> ręki <strona>."},
]


# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """
### Instruction
Przanalizuj medyczny tekst z Input.
Ważne infromacje: \
"""

# and the suffix our user input and output indicator
suffix = """
### Input
{ocr}

### Response
Znajdź w tekście odpowiedzi na poniższe pytania:
- Jaki jest numer palca którego dotyczy uraz, napisz słownie?
- Czy wystąpiło złamanie z przemieszczeniem ?
- Czy rana była otwarta ?
- Czy założono usztywnienie ?
- Czy złamanie jest jednoodłamowe czy wieloodłamowe?
- Czy uraz jest częściowy czy całkowity?

Odpowiedzi napisz najkrócej jak się da.
"""

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=knowledge_lst,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["ocr"],
    example_separator="\n"
)

ocr = df.iloc[0]["ocr"]
print(few_shot_prompt_template.format(ocr=ocr))


### Instruction
Przanalizuj medyczny tekst z Input.
Ważne infromacje: 
- Człowiek możne mieć jedną rękę lewą i jedną rękę prawą.
- Dokładny numer palca zapisuje się cyframi rzymskimi np. I, II , III, IV, V lub i, ii , iii, iv, v.
- Przykładowa lokalizacja palca to: <uraz> palca <cyfra rzymska> ręki <strona>.

### Input
z.i. iIIII- 1:; . LRTA.Un. g3-33 2995277 NIP: 6Błar: . Nr . ISJRÓYRMIAGYJNA IZBY PRZYJIć . Nazyiskć xAdres: . Dnia iodlz. przyjęcia: . 2020-04-10 15:17 . SPESEL: . Data urodzenia: . Pata iąodz wyypisu: 2020-04-10 15:40 . ŚOZPOZNANZA . SŚ.1 Otwarta rana palca Cpalców) z uszkodzeniem paznokcia . Gliwice, dnia OSod . Pacjent zgłosił się z powodu rany palca I ręki prawej - Rana powstała W wyniku kontaktu z piłą mechaniczng. . BADANIE PODMIOTOWE . Rana opuszki palca I ręki prawej wraz z częściowym uszkodzeniem paznokcia. w znieczulenfu miejscowym szycie rany   opatrunck, anatoksyna . Dystalna ruchomośc , ukrwienie i unerwienie zachowane, pełne a . K8rtrola w Poradni Cnirurgi

## określenie limitu tokenów? dla Example

In [12]:
### OKRŚLASZ MAX ILE SŁOW MOŻE MIEC TEN SEKTOR ! 
from langchain.prompts.example_selector import LengthBasedExampleSelector


example_selector = LengthBasedExampleSelector(
    examples=knowledge_lst,
    example_prompt=example_prompt,
    max_length=190  # this sets the max length that examples should be
)

# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["ocr"],
    example_separator="\n"
)

ocr = df.iloc[0]["ocr"]
print(dynamic_prompt_template.format(ocr=ocr))


### Instruction
Przanalizuj medyczny tekst z Input.
Ważne infromacje: 
- Człowiek możne mieć jedną rękę lewą i jedną rękę prawą.
- Dokładny numer palca zapisuje się cyframi rzymskimi np. I, II , III, IV, V lub i, ii , iii, iv, v.
- Przykładowa lokalizacja palca to: <uraz> palca <cyfra rzymska> ręki <strona>.

### Input
z.i. iIIII- 1:; . LRTA.Un. g3-33 2995277 NIP: 6Błar: . Nr . ISJRÓYRMIAGYJNA IZBY PRZYJIć . Nazyiskć xAdres: . Dnia iodlz. przyjęcia: . 2020-04-10 15:17 . SPESEL: . Data urodzenia: . Pata iąodz wyypisu: 2020-04-10 15:40 . ŚOZPOZNANZA . SŚ.1 Otwarta rana palca Cpalców) z uszkodzeniem paznokcia . Gliwice, dnia OSod . Pacjent zgłosił się z powodu rany palca I ręki prawej - Rana powstała W wyniku kontaktu z piłą mechaniczng. . BADANIE PODMIOTOWE . Rana opuszki palca I ręki prawej wraz z częściowym uszkodzeniem paznokcia. w znieczulenfu miejscowym szycie rany   opatrunck, anatoksyna . Dystalna ruchomośc , ukrwienie i unerwienie zachowane, pełne a . K8rtrola w Poradni Cnirurgi

# Dynamiczny prompt -   example_selector

In [14]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.llms import OpenAI
 

prefix = """
### Instruction
Przanalizuj medyczny tekst z Input.
Ważne infromacje: \
"""

suffix = """
### Input
{ocr}

### Response
Znajdź w tekście odpowiedzi na poniższe pytania:
- Jaki jest numer palca którego dotyczy uraz, napisz słownie?
- Czy wystąpiło złamanie z przemieszczeniem ?
- Czy rana była otwarta ?
- Czy założono usztywnienie ?
- Czy złamanie jest jednoodłamowe czy wieloodłamowe?
- Czy uraz jest częściowy czy całkowity?

Odpowiedzi napisz najkrócej jak się da.
"""


example_prompt = PromptTemplate(
    input_variables=["knowledge"],
    template="- {knowledge}"
)
 
# Examples that nouns are found
examples = [
    {"knowledge" : "Człowiek możne mieć jedną rękę lewą i jedną rękę prawą."}, 
    {"knowledge" : "Dokładny numer palca zapisuje się cyframi rzymskimi np. I, II , III, IV, V lub i, ii , iii, iv, v."},
    {"knowledge" : "Przykładowa lokalizacja palca to: <uraz> palca <cyfra rzymska> ręki <strona>."},
]

 
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    OpenAIEmbeddings(openai_api_key=openai.api_key),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    FAISS,
    # This is the number of examples to produce.
    k=2,
)
 
# example_selector - umiesznay jest na koniec prefix tak aby był pomiędzy prefix`em a suffix`em
similar_prompt = FewShotPromptTemplate(
    # The object that will help select examples
    example_selector=example_selector,
    # Your prompt
    example_prompt=example_prompt,
    # Customizations that will be added to the top and bottom of your prompt
    prefix=prefix,
    suffix=suffix,
    input_variables=["ocr"],
    example_separator="\n"
)

# Select a noun!
ocr = df.iloc[0]["ocr"]
print(similar_prompt.format(ocr=ocr))


### Instruction
Przanalizuj medyczny tekst z Input.
Ważne infromacje: 
- Przykładowa lokalizacja palca to: <uraz> palca <cyfra rzymska> ręki <strona>.
- Dokładny numer palca zapisuje się cyframi rzymskimi np. I, II , III, IV, V lub i, ii , iii, iv, v.

### Input
z.i. iIIII- 1:; . LRTA.Un. g3-33 2995277 NIP: 6Błar: . Nr . ISJRÓYRMIAGYJNA IZBY PRZYJIć . Nazyiskć xAdres: . Dnia iodlz. przyjęcia: . 2020-04-10 15:17 . SPESEL: . Data urodzenia: . Pata iąodz wyypisu: 2020-04-10 15:40 . ŚOZPOZNANZA . SŚ.1 Otwarta rana palca Cpalców) z uszkodzeniem paznokcia . Gliwice, dnia OSod . Pacjent zgłosił się z powodu rany palca I ręki prawej - Rana powstała W wyniku kontaktu z piłą mechaniczng. . BADANIE PODMIOTOWE . Rana opuszki palca I ręki prawej wraz z częściowym uszkodzeniem paznokcia. w znieczulenfu miejscowym szycie rany   opatrunck, anatoksyna . Dystalna ruchomośc , ukrwienie i unerwienie zachowane, pełne a . K8rtrola w Poradni Cnirurgicznej za dni . BADANIE IPRZRDMIOTOWE . WYNIRCI BADAŃ LABOR